In [ ]:
import sys
import time
import traceback
from arduino_iot_cloud import ArduinoCloudClient
import csv 
import plotly.io as pio
import plotly.express as px
import pandas as pd
import numpy as np
import dash
from dash import dcc, html, Patch
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go
import asyncio
import datetime
from datetime import datetime
from collections import deque

In [ ]:
DEVICE_ID = "d2d8a108-d4c3-4794-bf15-271885269210"
SECRET_KEY = "YyuZ6b3CQZHTJC#TGX@@H7AiP"

In [ ]:
arduino = ArduinoCloudClient(
            device_id=DEVICE_ID, username=DEVICE_ID, password=SECRET_KEY
        )

In [ ]:
# 2 buffer data one for temporary and the one for plotting
buffer_data = []
plot_data = []

#### Original use of Partial Property Update

In [ ]:
# Initialize Dash app
app = dash.Dash(__name__)

# Example initial plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=[], y=[], mode='lines', name='X'))
fig.add_trace(go.Scatter(x=[], y=[], mode='lines', name='Y'))
fig.add_trace(go.Scatter(x=[], y=[], mode='lines', name='Z'))

app.layout = html.Div([
    dcc.Graph(figure=fig, id='live-update-graph'),
    dcc.Interval(id='interval-component', interval=1000, n_intervals=0)  # Update every second
])

@app.callback(
    Output('live-update-graph', 'figure'),
    Input('interval-component', 'n_intervals')
)
def update_graph(n_intervals):
    global plot_data
    if plot_data:
        # Convert the plot_data to structured format for updating the graph
        reformatted_data = {
            'timestamp': [row[1] for row in plot_data],
            'x': [row[2] for row in plot_data],
            'y': [row[3] for row in plot_data],
            'z': [row[4] for row in plot_data]
        }
        # Update the graph with the new data
        patch = Patch()
        patch["data"] = [
            {"x": reformatted_data['timestamp'], "y": reformatted_data['x']},
            {"x": reformatted_data['timestamp'], "y": reformatted_data['y']},
            {"x": reformatted_data['timestamp'], "y": reformatted_data['z']}
        ]
        return patch
    return go.Figure()

In [ ]:
app.run_server(debug=True, port=8006)

In [ ]:

# Variables for data collection
x, y, z = 0, 0, 0
time = 0
num_threshold = 5
# Arduino Cloud Data Collection
def on_x_changed(client, value):
    global x
    x = value

def on_y_changed(client, value):
    global y
    y = value

def on_z_changed(client, value):
    global z
    z = value
if __name__ == "__main__":
    client = ArduinoCloudClient(device_id=DEVICE_ID, username=DEVICE_ID, 
                                password=SECRET_KEY, sync_mode=True)
    # Register the callback functions
    client.register("x", value=None, on_write=on_x_changed)
    client.register("y", value=None, on_write=on_y_changed)
    client.register("z", value=None, on_write=on_z_changed)
    client.start()
while True:
    # Check if x, y, and z are all recorded
    if x is not None and y is not None and z is not None:
        if time < num_threshold:
            time += 1
            current_timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            buffer_data.append([time, current_timestamp, x, y, z])
            print([time, current_timestamp, x, y, z])

            # Reset x, y, z for next readings
            x, y, z = None, None, None
        else:
            current_timestamp = datetime.now()
            format_time = current_timestamp.strftime('%Y-%m-%d-%H-%M-%S')
            df = pd.DataFrame(buffer_data, 
                              columns=['Index', 'Timestamp', 'X_Acce', 'Y_Acce', 'Z_Acce'])
            filename = f"csv_{format_time}.csv"
            #df.to_csv(filename)

            # Reset the time after reaching the threshold
            time = 0
            plot_data = buffer_data.copy()  # Update plot_data to be used in the Dash app

            # Clear the buffer for the next cycle
            buffer_data.clear()
    client.update()  # Sync the Arduino cloud client


#### Wrapping API function for later use 

In [ ]:
# API Function
def update_smoothly(data=None, x_name=None, y_name=None, dash_app=None, 
                    graph_id=None, interval_id=None):
    """
    Updates the graph in a Dash application smoothly using incremental updates.

    Parameters:
    - data (list of lists or DataFrame): Data to be used for updating the graph.
    - x_name (str): The column name in `data` to be used for the x-axis.
    - y_name (list of str): List of column names in `data` to be used for the y-axis.
    - dash_app (Dash): The Dash application instance where the callback will be registered.
    - graph_id (str): The ID of the graph component to be updated.
    - interval_id (str): The ID of the interval component triggering the updates.

    Raises:
    - ValueError: If any of the provided arguments are None.
    """
    
    # Define a dictionary of arguments for validation
    arguments = {
        "data": data,
        "x_name": x_name,
        "y_name": y_name,
        "dash_app": dash_app,
        "graph_id": graph_id,
        "interval_id": interval_id
    }
    
    # Validate that none of the arguments are None
    for arg_name, arg_value in arguments.items():
        if arg_value is None:
            raise ValueError(f"{arg_name} mustn't be None.")
    
    @dash_app.callback(
        Output(graph_id, 'figure'),
        Input(interval_id, 'n_intervals')
    )
    def update_graph(n_intervals):
        """
        Callback function to update the graph with new data.

        Parameters:
        - n_intervals (int): The number of intervals since the last update (used to trigger the update).

        Returns:
        - dict: A dictionary representing the updated figure with new data.
        """
        # Convert data to a DataFrame for easier manipulation
        df = pd.DataFrame(data, columns=['Index', 'Timestamp', 'X', 'Y', 'Z'])
        
        # Initialize a Patch object to handle incremental updates
        patch = Patch()
        
        # Update the graph data with the provided y_names
        for i in range(len(y_name)):
            patch["data"][i] = {"x": df[x_name], "y": df[y_name[i]]}
        
        return patch


In [ ]:
# Initialize Dash app
# Begin the function alternative the one used in task 8.1P
app = dash.Dash(__name__)

fig = go.Figure()
fig.add_trace(go.Scattergl(x=[], y=[], mode='lines', name='X_Acce'))
fig.add_trace(go.Scattergl(x=[], y=[], mode='lines', name='Y_Acce'))
fig.add_trace(go.Scattergl(x=[], y=[], mode='lines', name='Z_Acce'))

app.layout = html.Div([
    dcc.Graph(figure=fig, id='update-graph'),
    dcc.Interval(id='interval-component', interval= 1000, n_intervals=0)  # Update every second
])
update_smoothly(data = plot_data, x_name = 'Timestamp' , y_name = ['X', 'Y', 'Z'] , 
                dash_app = app , graph_id = 'update-graph' , interval_id = 'interval-component')
app.run_server(debug=True, port=8003)

In [ ]:
# Variables for data collection
x, y, z = 0, 0, 0
time = 0
num_threshold = 5

# Arduino Cloud Data Collection
def on_x_changed(client, value):
    global x
    x = value

def on_y_changed(client, value):
    global y
    y = value

def on_z_changed(client, value):
    global z
    z = value

if __name__ == "__main__":
    client = ArduinoCloudClient(device_id=DEVICE_ID, username=DEVICE_ID, 
                                password=SECRET_KEY, sync_mode=True)
    # Register the callback functions
    client.register("x", value=None, on_write=on_x_changed)
    client.register("y", value=None, on_write=on_y_changed)
    client.register("z", value=None, on_write=on_z_changed)
    client.start()

    while True:
        # Check if x, y, and z are all recorded
        if x is not None and y is not None and z is not None:
            if time < num_threshold:
                time += 1
                current_timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                buffer_data.append([time, current_timestamp, x, y, z])
                print([time, current_timestamp, x, y, z])

                # Reset x, y, z for next readings
                x, y, z = None, None, None
            else:
                current_timestamp = datetime.now()
                format_time = current_timestamp.strftime('%Y-%m-%d-%H-%M-%S')

                # Reset the time after reaching the threshold
                time = 0
                plot_data = buffer_data.copy()  # Update plot_data to be used in the Dash app
                df = pd.DataFrame(plot_data, columns = ['Index','Timestamp', 'X',  'Y', 'Z'])
                print(plot_data)
                #update_smoothly(data = plot_data, x_name = 'Timestamp' , y_name = ['X', 'Y', 'Z'] , 
                #dash_app = app , graph_id = 'update-graph' , interval_id = 'interval-component')
                # Clear the buffer for the next cycle
                buffer_data.clear()
        client.update()  # Sync the Arduino cloud client
